In [1]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime

In [2]:
link = 'http://www.justificando.com/categoria/noticias/'

In [3]:
req = requests.get(link)

In [4]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('h4', class_='entry-title entry-title-left')

In [5]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    article = NewsPlease.from_url(noticia.find_all('a', href=True)[0]['href'])
    row = {'titulos': [], 'links': [], 'noticia': [], 'image': [], 'abstract': [], 'date': []}
    if (article is not None):
        row['titulos'].append(article.title)
        row['noticia'].append(article.text)
        row['links'].append(article.url)
        row['abstract'].append(article.text)
        row['date'].append(article.date_publish)
        path_image = article.image_url
        print(path_image)
        if path_image == '' or path_image == None:
            row['image'].append(0)
        else:
            row['image'].append(download_and_move_image(article.image_url))
        news = News(row['abstract'], row['noticia'], row['date'], row['links'], row['titulos'], row['image'])
        try:
            print(row['titulos'])
            news_in_db = midia_table.check_news(news)
            print('news_in_db: ' + str(news_in_db))
            if (not news_in_db):
                row = pd.DataFrame(row)
                df, categories = midia_lexical.lexical_corpus_and_title(row)
                # DB categories
                if (categories != [set()]):
                    news.set_categories(categories)
                    midia_table.save_news(news)
                    midia_post.post_news(df)
        except:
            print('Empty News')

http://www.justificando.com/2018/10/11/ministerio-publico-abre-inquerito-contra-o-game-bolsomito-2k18/
http://www.justificando.com/wp-content/uploads/2018/10/bolsojogo-redes.png
HTTP Error 403: Forbidden
['Ministério Público abre inquérito contra o game Bolsomito 2k18']
news_in_db: False
 -- no categories -- 
http://www.justificando.com/2018/10/11/e-10-dias-50-ataques-provocados-por-bolsonaristas/
http://www.justificando.com/wp-content/uploads/2018/10/Casos-de-violência-2.png
HTTP Error 403: Forbidden
['Em 10 dias, 50 ataques provocados por bolsonaristas']
news_in_db: False
 -- no categories -- 
http://www.justificando.com/2018/10/11/bolsonaristas-marcam-pele-de-mulher-com-suastica-apos-agredi-la-com-socos/
http://www.justificando.com/wp-content/uploads/2018/10/Bolsonaristas-marcam-pele-de-mulher-com-suástica-após-agredi-la-com-socos-2.png
HTTP Error 403: Forbidden
['Bolsonaristas marcam pele de mulher com suástica após agredi-la com socos']
news_in_db: False
 -- no categories -- 
h

In [6]:
noticias[0]

<h4 class="entry-title entry-title-left"><a href="http://www.justificando.com/2018/10/11/ministerio-publico-abre-inquerito-contra-o-game-bolsomito-2k18/" title="Ministério Público abre inquérito contra o game Bolsomito 2k18">Ministério Público abre inquérito contra o game Bolsomito 2k18</a></h4>